In [1]:
#!/usr/bin/python3
'''Prepare Data for Amsterdam Segmentation Task.'''

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import math
import h5py
import argparse
import numpy as np
from datetime import datetime
import liblas

import data_utils

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [169]:
args = argparse.Namespace(
    folder = None,
    max_point_num = 24576,#8192,
    save_ply = False,#True,
    # meters
    block_size = 50.0,#5.0,
    grid_size = 1.0,#0.1,
    # feet
#     block_size = 150.0,#5.0,
#     grid_size = 3.0,#0.1,
)

In [170]:

# parser = argparse.ArgumentParser()
# parser.add_argument('--folder', '-f', help='Path to data folder')
# parser.add_argument('--max_point_num', '-m', help='Max point number of each sample', type=int, default=8192)
# parser.add_argument('--block_size', '-b', help='Block size', type=float, default=5.0)
# parser.add_argument('--grid_size', '-g', help='Grid size', type=float, default=0.1)
# parser.add_argument('--save_ply', '-s', help='Convert .pts to .ply', action='store_true')

# args = parser.parse_args()
# print(args)

root = args.folder if args.folder else '../data/Amsterdam/'
max_point_num = args.max_point_num

batch_size = 2048 #how many blocks of block_size * block_size with max_point_num points are written in a single H5
data = np.zeros((batch_size, max_point_num, 3))
data_num = np.zeros((batch_size), dtype=np.int32)
label = np.zeros((batch_size), dtype=np.int32)
label_seg = np.zeros((batch_size, max_point_num), dtype=np.int32)
indices_split_to_full = np.zeros((batch_size, max_point_num), dtype=np.int32)

if args.save_ply:
    data_center = np.zeros((batch_size, max_point_num, 3))


In [171]:
print('data.shape: {}'.format(data.shape))
print('data_num.shape: {}'.format(data_num.shape))
print('label.shape: {}'.format(label.shape))
print('label_seg.shape: {}'.format(label_seg.shape))
print('indices_split_to_full.shape: {}'.format(indices_split_to_full.shape))


data.shape: (2048, 24576, 3)
data_num.shape: (2048,)
label.shape: (2048,)
label_seg.shape: (2048, 24576)
indices_split_to_full.shape: (2048, 24576)


In [172]:
def read_xyz_label_from_txt(filename_txt):
    print('{}-Loading {}...'.format(datetime.now(), filename_txt))
    xyzirgb = np.loadtxt(filename_txt)
    xyzirgb_num = xyzirgb.shape[0]
    print('Number of records: {}'.format(xyzirgb_num))
    xyz, labels = np.split(xyzirgb, [3], axis=-1)
    labels = labels.flatten()
    return xyz, labels, xyzirgb_num
    
def read_xyz_label_from_las(filename_las):
    print('{}-Loading {}...'.format(datetime.now(), filename_las))
    f = liblas.file.File(filename_las, mode='r')    
    h = f.header
    xyzirgb_num = h.point_records_count
    xyz = np.ndarray((xyzirgb_num, 3))
    labels = np.ndarray(xyzirgb_num, np.int16)
    i = 0
    for p in f:
#         xyz[i] = [p.raw_x, p.raw_y, p.raw_z] 
        xyz[i] = [p.x, p.y, p.z] 
        labels[i] = p.classification
        i += 1
    print('Number of records: {}'.format(xyzirgb_num))
    return xyz, labels, xyzirgb_num

def save_xyz_label_to_las(filename_las, xyz, labels):  
#     msg = 'Saving {}...'.format(filename_las)
#     timer_start(msg)
    h = liblas.header.Header()
    h.dataformat_id = 1
    h.major = 1
    h.minor = 2
    h.min = np.min(xyz, axis=0)
    h.max = np.max(xyz, axis=0)
    h.scale = [1e-3, 1e-3, 1e-3]
    
    f = liblas.file.File(filename_las, mode='w', header=h)    
    for i in range(xyz.shape[0]):
        p = liblas.point.Point()
        p.x = xyz[i,0] / h.scale[0]
        p.y = xyz[i,1] / h.scale[1]
        p.z = xyz[i,2] / h.scale[2]
        p.classification = labels[i]
        p.color = liblas.color.Color()
        p.intensity = 100
        p.return_number = 1
        p.number_of_returns = 1
        p.scan_direction = 1
        p.scan_angle = 0
        f.write(p)
    f.close()
#     timer_stop(msg)

In [174]:
LOAD_FROM_EXT = '.las'

folders = [os.path.join(root, folder) for folder in ['Utrecht-test-2']]#['train', 'val']]#, 'test']]
for folder in folders:
    datasets = [filename[:-4] for filename in os.listdir(folder) if filename.endswith(LOAD_FROM_EXT)]
    print(folder)
    for dataset_idx, dataset in enumerate(datasets):

        filename_ext = os.path.join(folder, dataset + LOAD_FROM_EXT)
        if LOAD_FROM_EXT == '.las':
            xyz, labels, xyz_num = read_xyz_label_from_las(filename_ext)
        else:
            xyz, labels, xyz_num = read_xyz_label_from_txt(filename_ext)

        offsets = [('zero', 0.0), ('half', args.block_size / 2)]

        for offset_name, offset in offsets:
            idx_h5 = 0
            idx = 0

            print('{}-Computing block id of {} points...'.format(datetime.now(), xyz_num))
            xyz_min = np.amin(xyz, axis=0, keepdims=True) - offset
            xyz_max = np.amax(xyz, axis=0, keepdims=True)
            block_size = (args.block_size, args.block_size, 2 * (xyz_max[0, -1] - xyz_min[0, -1]))
            xyz_blocks = np.floor((xyz - xyz_min) / block_size).astype(np.int)

            print('{}-Collecting points belong to each block...'.format(datetime.now(), xyz_num))
            blocks, point_block_indices, block_point_counts = np.unique(xyz_blocks, return_inverse=True,
                                                                        return_counts=True, axis=0)
            block_point_indices = np.split(np.argsort(point_block_indices), np.cumsum(block_point_counts[:-1]))
            print('{}-{} is split into {} blocks.'.format(datetime.now(), dataset, blocks.shape[0]))

            block_to_block_idx_map = dict()
            for block_idx in range(blocks.shape[0]):
                block = (blocks[block_idx][0], blocks[block_idx][1])
                block_to_block_idx_map[(block[0], block[1])] = block_idx

            # merge small blocks into one of their big neighbors
            block_point_count_threshold = max_point_num / 10
            nbr_block_offsets = [(0, 1), (1, 0), (0, -1), (-1, 0), (-1, 1), (1, 1), (1, -1), (-1, -1)]
            block_merge_count = 0
            for block_idx in range(blocks.shape[0]):
                if block_point_counts[block_idx] >= block_point_count_threshold:
                    continue

                block = (blocks[block_idx][0], blocks[block_idx][1])
                for x, y in nbr_block_offsets:
                    nbr_block = (block[0] + x, block[1] + y)
                    if nbr_block not in block_to_block_idx_map:
                        continue

                    nbr_block_idx = block_to_block_idx_map[nbr_block]
                    if block_point_counts[nbr_block_idx] < block_point_count_threshold:
                        continue

                    block_point_indices[nbr_block_idx] = np.concatenate(
                        [block_point_indices[nbr_block_idx], block_point_indices[block_idx]], axis=-1)
                    block_point_indices[block_idx] = np.array([], dtype=np.int)
                    block_merge_count = block_merge_count + 1
                    break
            print('{}-{} of {} blocks are merged.'.format(datetime.now(), block_merge_count, blocks.shape[0]))

            idx_last_non_empty_block = 0
            for block_idx in reversed(range(blocks.shape[0])):
                if block_point_indices[block_idx].shape[0] != 0:
                    idx_last_non_empty_block = block_idx
                    break

            # uniformly sample each block
            for block_idx in range(idx_last_non_empty_block + 1):
                point_indices = block_point_indices[block_idx]
                if point_indices.shape[0] == 0:
                    continue
                block_points = xyz[point_indices]
                block_min = np.amin(block_points, axis=0, keepdims=True)
                xyz_grids = np.floor((block_points - block_min) / args.grid_size).astype(np.int)
                grids, point_grid_indices, grid_point_counts = np.unique(xyz_grids, return_inverse=True,
                                                                         return_counts=True, axis=0)
                grid_point_indices = np.split(np.argsort(point_grid_indices), np.cumsum(grid_point_counts[:-1]))
                grid_point_count_avg = int(np.average(grid_point_counts))
                point_indices_repeated = []
                for grid_idx in range(grids.shape[0]):
                    point_indices_in_block = grid_point_indices[grid_idx]
                    repeat_num = math.ceil(grid_point_count_avg / point_indices_in_block.shape[0])
                    if repeat_num > 1:
                        point_indices_in_block = np.repeat(point_indices_in_block, repeat_num)
                        np.random.shuffle(point_indices_in_block)
                        point_indices_in_block = point_indices_in_block[:grid_point_count_avg]
                    point_indices_repeated.extend(list(point_indices[point_indices_in_block]))
                block_point_indices[block_idx] = np.array(point_indices_repeated)
                block_point_counts[block_idx] = len(point_indices_repeated)
            ''' - '''
            for block_idx in range(idx_last_non_empty_block + 1):
                point_indices = block_point_indices[block_idx]
                if point_indices.shape[0] == 0:
                    continue

                block_point_num = point_indices.shape[0]
                block_split_num = int(math.ceil(block_point_num * 1.0 / max_point_num))
                point_num_avg = int(math.ceil(block_point_num * 1.0 / block_split_num))
                point_nums = [point_num_avg] * block_split_num
                point_nums[-1] = block_point_num - (point_num_avg * (block_split_num - 1))
                starts = [0] + list(np.cumsum(point_nums))

                np.random.shuffle(point_indices)
                block_points = xyz[point_indices]
                block_min = np.amin(block_points, axis=0, keepdims=True)
                block_max = np.amax(block_points, axis=0, keepdims=True)
                block_center = (block_min + block_max) / 2
                block_center[0][-1] = block_min[0][-1]
                block_points = block_points - block_center  # align to block bottom center
                x, y, z = np.split(block_points, (1, 2), axis=-1)
#                 block_xzyrgbi = np.concatenate([x, z, y, rgb[point_indices], i[point_indices]], axis=-1)
                block_xzyrgbi = np.concatenate([x, z, y], axis=-1)
                block_labels = labels[point_indices]

                for block_split_idx in range(block_split_num):
                    start = starts[block_split_idx]
                    point_num = point_nums[block_split_idx]
                    end = start + point_num
                    idx_in_batch = idx % batch_size
                    data[idx_in_batch, 0:point_num, ...] = block_xzyrgbi[start:end, :]
                    data_num[idx_in_batch] = point_num
                    label[idx_in_batch] = dataset_idx  # won't be used...
                    label_seg[idx_in_batch, 0:point_num] = block_labels[start:end]
                    indices_split_to_full[idx_in_batch, 0:point_num] = point_indices[start:end]
                    if args.save_ply:
                        block_center_xzy = np.array([[block_center[0][0], block_center[0][2], block_center[0][1]]])
                        data_center[idx_in_batch, 0:point_num, ...] = block_center_xzy

                    if ((idx + 1) % batch_size == 0) or \
                            (block_idx == idx_last_non_empty_block and block_split_idx == block_split_num - 1):
                        item_num = idx_in_batch + 1
                        filename_h5 = os.path.join(folder, dataset + '_%s_%d.h5' % (offset_name, idx_h5))
                        print('{}-Saving {}...'.format(datetime.now(), filename_h5))

                        file = h5py.File(filename_h5, 'w')
                        file.create_dataset('data', data=data[0:item_num, ...])
                        file.create_dataset('data_num', data=data_num[0:item_num, ...])
                        file.create_dataset('label', data=label[0:item_num, ...])
                        file.create_dataset('label_seg', data=label_seg[0:item_num, ...])
                        file.create_dataset('indices_split_to_full', data=indices_split_to_full[0:item_num, ...])
                        file.close()

                        if args.save_ply and offset_name == 'zero':
                            print('{}-Saving ply of {}...'.format(datetime.now(), filename_h5))
                            filepath_label_ply = os.path.join(folder, 'ply_label',
                                                              dataset + '_label_%s_%d' % (offset_name, idx_h5))
                            data_utils.save_ply_property_batch(
                                data[0:item_num, :, 0:3] + data_center[0:item_num, ...],
                                label_seg[0:item_num, ...],
                                filepath_label_ply, data_num[0:item_num, ...], 8)

#                             filepath_label_aligned_ply = os.path.join(folder, 'ply_label_aligned',
#                                                                       dataset + '_label_%s_%d' % (
#                                                                           offset_name, idx_h5))
#                             data_utils.save_ply_property_batch(data[0:item_num, :, 0:3],
#                                                                label_seg[0:item_num, ...],
#                                                                filepath_label_aligned_ply,
#                                                                data_num[0:item_num, ...], 8)

                        idx_h5 = idx_h5 + 1
                    idx = idx + 1

print('{}-Done.'.format(datetime.now()))


../data/Amsterdam/Utrecht-test-2
2019-03-08 14:21:51.553881-Loading ../data/Amsterdam/Utrecht-test-2/Tile_136500_450500.las...
Number of records: 4692112
2019-03-08 14:22:35.782324-Computing block id of 4692112 points...
2019-03-08 14:22:36.186378-Collecting points belong to each block...
2019-03-08 14:22:40.864713-Tile_136500_450500 is split into 100 blocks.
2019-03-08 14:22:40.865808-3 of 100 blocks are merged.
2019-03-08 14:22:49.692772-Saving ../data/Amsterdam/Utrecht-test-2/Tile_136500_450500_zero_0.h5...
2019-03-08 14:22:49.825205-Computing block id of 4692112 points...
2019-03-08 14:22:50.218071-Collecting points belong to each block...
2019-03-08 14:22:55.058390-Tile_136500_450500 is split into 119 blocks.
2019-03-08 14:22:55.059678-4 of 119 blocks are merged.
2019-03-08 14:23:03.775160-Saving ../data/Amsterdam/Utrecht-test-2/Tile_136500_450500_half_0.h5...
2019-03-08 14:23:03.906781-Loading ../data/Amsterdam/Utrecht-test-2/Tile_136000_453000.las...
Number of records: 4391530
2

### DEBUG

In [ ]:
 xyz, labels = np.split(xyzirgb, [3], axis=-1)

In [ ]:
block_labels[start:end].shape

In [ ]:
label_seg[idx_in_batch, 0:point_num].shape

In [ ]:
labels.flatten().shape